In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://localhost:8888") \
    .appName("spark_job") \
    .config("spark.sql.autoBroadcastJoinThreshold", '-1') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 21:02:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
match_details = spark.read.csv("../../data/match_details.csv", header=True, inferSchema=True)
matches = spark.read.csv("../../data/matches.csv", header=True, inferSchema=True)
medal_matches_players = spark.read.csv("../../data/medals_matches_players.csv", header=True, inferSchema=True)

In [5]:
with open("sql/ddl_match_details_bucketed.sql", "r") as f:
    ddl_match_details = f.read()


In [6]:
print(ddl_match_details)

CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id string,
    player_gamertag string,
    previous_spartan_rank integer,
    spartan_rank integer,
    previous_total_xp integer,
    total_xp integer,
    previous_csr_tier integer,
    previous_csr_designation integer,
    previous_csr integer,
    previous_csr_percent_to_next_tier integer,
    previous_csr_rank integer,
    current_csr_tier integer,
    current_csr_designation integer,
    current_csr integer,
    current_csr_percent_to_next_tier integer,
    current_csr_rank integer,
    player_rank_on_team integer,
    player_finished boolean,
    player_average_life string,
    player_total_kills integer,
    player_total_headshots integer,
    player_total_weapon_damage double,
    player_total_shots_landed integer,
    player_total_melee_kills integer,
    player_total_melee_damage double,
    player_total_assassinations integer,
    player_total_ground_pound_kills integer,
    player_total_shoulder_bash_kills in

In [7]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.match_details_bucketed""")
spark.sql(ddl_match_details)

24/12/05 20:35:15 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`bootcamp`.`matches_bucketed`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore


root
 |-- medal_id: long (nullable = true)
 |-- sprite_uri: string (nullable = true)
 |-- sprite_left: integer (nullable = true)
 |-- sprite_top: integer (nullable = true)
 |-- sprite_sheet_width: integer (nullable = true)
 |-- sprite_sheet_height: integer (nullable = true)
 |-- sprite_width: integer (nullable = true)
 |-- sprite_height: integer (nullable = true)
 |-- classification: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- difficulty: integer (nullable = true)

root
 |-- mapid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)



In [16]:
medals.show()

+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|  medal_id|          sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|   classification|         description|          name|difficulty|
+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|2315448068|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|3565441934|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|4162659350|https://content.h...|        750|       750|    

In [17]:
# Count the number of rows per name in the medals dataframe and sort the result by the name
medals.groupBy("classification").count().sort("classification").show()

+-----------------+-----+
|   classification|count|
+-----------------+-----+
|             NULL|    2|
|             Ball|    8|
|         Breakout|   10|
|   CaptureTheFlag|   13|
|     KillingSpree|    9|
|        MultiKill|   11|
|      Strongholds|    5|
|            Style|   51|
|         Vehicles|   24|
|          Warzone|   18|
|WeaponProficiency|   32|
+-----------------+-----+



In [15]:
maps.sort("name").show()

+--------------------+------------------+--------------------+
|               mapid|              name|         description|
+--------------------+------------------+--------------------+
|cc74f4e1-f206-11e...|              NULL|                NULL|
|ce89a40f-f206-11e...|              NULL|                NULL|
|5e130537-2275-40b...|              NULL|                NULL|
|9a188f67-1664-4d7...|          Alliance|                NULL|
|c74c9d0f-f206-11e...|            Alpine|These vistas are ...|
|cae999f0-f206-11e...|             Array|Halo erased a hun...|
|c96ff240-f206-11e...|  Battle of Noctus|The Andesian gove...|
|8d72dd82-3afb-409...| Battle of Sunaion|                NULL|
|2702ea83-2c3e-4fd...|  Before the Storm|                NULL|
|96c3e3dd-7703-408...|         Blue Team|                NULL|
|c7edbf0f-f206-11e...|    Breakout Arena|The broadcast of ...|
|cebd854f-f206-11e...|          Coliseum|Forerunner Warrio...|
|c89dae21-f206-11e...|          Dispatch|              